### PoC Satellite Imagery for Road quality assessments

Install

In [ ]:
#!pip install geoplot
#!pip install geojsonio
#!pip install up42-py
#!pip install ipyleaflet

Import

In [82]:
import pandas as pd
import geopandas as gpd
import folium
import up42
import os

### Catalog search: Area of Interest (aoi)

Read road alignment file and check crs

In [132]:
# Read road alignment 
road_alignment = gpd.read_file('./road_alignment.geojson')

In [133]:
# Check Coordinate Reference Systems
road_alignment.crs

<Derived Projected CRS: EPSG:32636>
Name: WGS 84 / UTM zone 36N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 30°E and 36°E, northern hemisphere between equator and 84°N, onshore and offshore. Belarus. Cyprus. Egypt. Ethiopia. Finland. Israel. Jordan. Kenya. Lebanon. Moldova. Norway. Russian Federation. Saudi Arabia. Sudan. Syria. Turkey. Uganda. Ukraine.
- bounds: (30.0, 0.0, 36.0, 84.0)
Coordinate Operation:
- name: UTM zone 36N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [134]:
# Transform coordinate system
road_alignment = road_alignment.to_crs('EPSG:4326')

Buffer around road alignment

In [135]:
# Length of one degree of longitude at the equator (in kilometers)
length_of_degree_at_equator = 111.111  # Approximately 111 kilometers

# Desired buffer width (in meters)
corridor_wide = 500.0  # 500 meters

# Convert desired buffer width from meters to kilometers
corridor_wide_km = corridor_wide / 1000

# Calculate buffer distance width in degrees
aoi_buffer_degrees = (corridor_wide_km / length_of_degree_at_equator)/2

# Output the buffer distance in degrees
print("Buffer distance in degrees:", aoi_buffer_degrees)


Buffer distance in degrees: 0.00225000225000225


In [148]:
# Business Specification: corridor that is 500m wide (250m buffer)

# Create a buffer around the road
buffer_distance = aoi_buffer_degrees  
road_alignment_buffer = road_alignment.buffer(buffer_distance)

# Convert GeoDataFrame to GeoJSON object
aoi = road_alignment_buffer.to_json()
geojson_data_line = road_alignment.to_json()

C:\Users\Nisleida Morales\AppData\Local\Temp\ipykernel_25148\3842963363.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  road_alignment_buffer = road_alignment.buffer(buffer_distance)


Visualize aoi

In [149]:
# Create a base map
m = folium.Map(location=[road_alignment.geometry.centroid.y.mean(), road_alignment.geometry.centroid.x.mean()], zoom_start=10)

# Add the buffer layer to the map
folium.GeoJson(aoi, name='Buffer').add_to(m)

# Add the road line layer to the map
folium.GeoJson(geojson_data_line, name='Road Alignment').add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Show the map
m

C:\Users\Nisleida Morales\AppData\Local\Temp\ipykernel_25148\898948169.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[road_alignment.geometry.centroid.y.mean(), road_alignment.geometry.centroid.x.mean()], zoom_start=10)
C:\Users\Nisleida Morales\AppData\Local\Temp\ipykernel_25148\898948169.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[road_alignment.geometry.centroid.y.mean(), road_alignment.geometry.centroid.x.mean()], zoom_start=10)


Save aoi as geojson

In [150]:
# Convert the buffer to a GeoDataFrame
road_alignment_buffer_gdf = gpd.GeoDataFrame(geometry=road_alignment_buffer, crs='EPSG:4326')

# Save the buffer as a GeoJSON file
aoi = 'road_alignment_buffer.geojson'
road_alignment_buffer_gdf.to_file(aoi, driver='GeoJSON')

## Catalogs search: available catalogs with UP42 Python SDK

Authentication in up42

In [87]:
# Authenticate with up42 credentials file. Register: https://console.up42.com/register

# SDK documentation https://sdk.up42.com/

def authenticate_with_credentials_file(credentials_file="credentials.json"):

    # Check if the credentials file exists
    if os.path.isfile(credentials_file):
        try:
            up42.authenticate(cfg_file=credentials_file)
            print("Authenticated 👍")
        except Exception as e:
            print("not authenticated:", e)
    else:
        print("Credentials file not found")


authenticate_with_credentials_file()

2024-02-24 10:36:32,464 - Authentication with UP42 successful!


Authenticated 👍


Construct a DataFrame containing data from all catalogs

In [44]:
# Initialize Catalog
catalog = up42.initialize_catalog()

# Available data collections
collection_list = catalog.get_collections()

# Convert list of dictionaries to DataFrame
collection_df = pd.DataFrame(collection_list)

# Display DataFrame
collection_df.head()

,name,title,description,type,restricted,host,hostName,producer,producerName,isIntegrated,isOptical,resolutionClass,productType,resolutionValue,createdAt,updatedAt
0,pneo,Pléiades Neo,Very high resolution 30 cm optical satellite i...,ARCHIVE,False,"{'name': 'oneatlas', 'title': 'OneAtlas', 'des...",oneatlas,"{'name': 'airbus', 'title': 'Airbus', 'descrip...",airbus,True,True,VERY_HIGH,OPTICAL,"{'minimum': 0.3, 'maximum': None}",2022-04-21T13:01:00.656522Z,2023-10-05T15:33:16.486420Z
1,landsat-8,Landsat 8,Free low resolution 30 m optical and thermal s...,ARCHIVE,False,"{'name': 'earthsearch-aws', 'title': 'Earth Se...",earthsearch-aws,"{'name': 'usgs', 'title': 'USGS', 'description...",usgs,True,True,LOW,OPTICAL,"{'minimum': 30, 'maximum': None}",2023-11-14T09:48:20.054738Z,2023-11-15T10:00:27.770645Z
2,hexagon-na-30cm,Hexagon — North America 30 cm,Very high resolution 30 cm optical aerial imag...,ARCHIVE,False,"{'name': 'hexagon', 'title': 'Hexagon', 'descr...",hexagon,"{'name': 'hexagon', 'title': 'Hexagon', 'descr...",hexagon,True,True,VERY_HIGH,OPTICAL,"{'minimum': 0.3, 'maximum': None}",2023-03-22T12:41:28Z,2023-10-05T15:43:33.108568Z
3,hexagon-na-15cm,Hexagon — North America 15 cm,Very high resolution 15 cm optical aerial imag...,ARCHIVE,False,"{'name': 'hexagon', 'title': 'Hexagon', 'descr...",hexagon,"{'name': 'hexagon', 'title': 'Hexagon', 'descr...",hexagon,True,True,VERY_HIGH,OPTICAL,"{'minimum': 0.3, 'maximum': None}",2023-03-22T12:41:28Z,2023-10-05T15:43:53.795986Z
4,vexcel-us-6cm-oblique,Vexcel Aerial Oblique — US 5.5 cm,Very high resolution 5.5 cm optical aerial ima...,ARCHIVE,False,"{'name': 'vexcel', 'title': 'Vexcel', 'descrip...",vexcel,"{'name': 'vexcel', 'title': 'Vexcel', 'descrip...",vexcel,True,True,VERY_HIGH,OPTICAL,"{'minimum': 0.055, 'maximum': None}",2023-06-14T01:32:39Z,2023-10-05T15:50:15.095888Z


In [88]:
# Check catalogs with minimum resolution according to business specification
# Business Specification: The preference is for 30cm data where possible and filling in with 50cm when needed.

#Create a column with the value as numeric of resolution in centimeters (now is in meters)
def min_resolution(row):
    min_value = row['resolutionValue']['minimum']  # Minimum value of resolution
    if min_value is not None:
        return min_value * 100 # Convert to cm
    else:
        return None

# Create new numeric column for min_resolution_cm
collection_df['min_resolution_cm'] = collection_df.apply(min_resolution, axis=1).apply(pd.to_numeric, errors='coerce')

Explore the Catalogs information

In [73]:
print(f"Collections attributes: \n {list(collection_df.columns)}\n")

print(f"Collection size: {len(collection_df)}\n")

print(f"Unique product type: \n {collection_df['productType'].unique()}\n")

print(f"Unique host: \n {list(collection_df['hostName'].unique())}\n")

print(f"Top min resolution of all catalogs: {collection_df['min_resolution_cm'].max()}\n")

print(f"Count catalogs with resolution less than 50 cm: {(collection_df['min_resolution_cm']<= 50).sum()}\n")

Collections attributes: 
 ['name', 'title', 'description', 'type', 'restricted', 'host', 'hostName', 'producer', 'producerName', 'isIntegrated', 'isOptical', 'resolutionClass', 'productType', 'resolutionValue', 'createdAt', 'updatedAt', 'min_resolution_cm']

Collection size: 51

Unique product type: 
 ['OPTICAL' 'ELEVATION' 'SAR']

Unique host: 
 ['oneatlas', 'earthsearch-aws', 'hexagon', 'vexcel', 'intermap', 'capellaspace', 'airbus-elevation', '21at', 'nearspacelabs', 'imagesat']

Top min resolution of all catalogs: 3000.0

Count catalogs with resolution less than 50 cm: 37



Filter catalogs

In [80]:
filtered_collections_df = collection_df[(collection_df['min_resolution_cm'] <= 50.0) & # less than 50 cm of min resolution
                            (collection_df['productType'] != 'ELEVATION') & # not elevation type 
                            (~collection_df['restricted'])] # be unrestricted

# Check remaining number of catalogs
len(filtered_collections_df)

22

In [125]:
catalog_names_array = filtered_collections_df['name'].values
print(f"💼 Catalogs according Business Specification: \n preference 30cm data or with 50cm when needed \n {catalog_names_array}")

💼 Catalogs according Business Specification: 
 preference 30cm data or with 50cm when needed 
 ['pneo' 'hexagon-na-30cm' 'hexagon-na-15cm' 'vexcel-us-6cm-oblique'
 'vexcel-oc-6cm-trueortho' 'vexcel-ca-6cm-trueortho' 'phr'
 'hexagon-eu-30cm' 'vexcel-eeu-6cm-trueortho' 'nsl-30cm'
 'vexcel-ca-6cm-oblique' 'vexcel-jp-6cm-oblique' 'vexcel-oc-6cm-oblique'
 'vexcel-eeu-6cm-oblique' 'vexcel-us-15cm-ortho' 'vexcel-jp-6cm-trueortho'
 'vexcel-weu-6cm-trueortho' 'vexcel-us-6cm-trueortho'
 'vexcel-weu-6cm-oblique' 'eros-b' 'vexcel-weu-15cm-ortho' 'eros-c']


## Catalog data search

In [151]:
# Read aio
aoi = up42.read_vector_file(aoi, as_dataframe=False)

Define range date

In [111]:
start_date_search="2023-05-01"
end_date_search="2023-09-30"
max_cloudcover=20
limit=10
print(f"💼 Date range according Business Specification: from {start_date} to {end_date}")

💼 Date range according Business Specification: from 2023-05-01 to 2023-09-30


In [ ]:
['pneo' 'hexagon-na-30cm' 'hexagon-na-15cm' 'vexcel-us-6cm-oblique'
 'vexcel-oc-6cm-trueortho' 'vexcel-ca-6cm-trueortho' 'phr'
 'hexagon-eu-30cm' 'vexcel-eeu-6cm-trueortho' 'nsl-30cm'
 'vexcel-ca-6cm-oblique' 'vexcel-jp-6cm-oblique' 'vexcel-oc-6cm-oblique'
 'vexcel-eeu-6cm-oblique' 'vexcel-us-15cm-ortho' 'vexcel-jp-6cm-trueortho'
 'vexcel-weu-6cm-trueortho' 'vexcel-us-6cm-trueortho'
 'vexcel-weu-6cm-oblique' 'eros-b' 'vexcel-weu-15cm-ortho' 'eros-c']

In [164]:
for i in catalog_names_array:
    search_parameters = catalog.construct_search_parameters(
        collections=[i], #name
        geometry = aoi,
        start_date=start_date_search,
        end_date=end_date_search,
        max_cloudcover=20,
        limit=10,
    )
    search_results_json = catalog.search((search_parameters), as_dataframe=False)
    print(f"catalog name: {str(i)} \n")
    #search_results_df = catalog.search((search_parameters), as_dataframe=True)

2024-02-24 11:43:58,503 - Searching catalog with search_parameters: {'datetime': '2023-05-01T00:00:00Z/2023-09-30T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((30.561470275866256, 40.7576736463021), (30.71621035278162, 40.792097518301624), (30.738823082999087, 40.79860291607337), (30.747175018694804, 40.80533377950656), (30.747363026007253, 40.805469917762096), (30.747564058842183, 40.80558596160483), (30.747775977603993, 40.80568067597832), (30.747996526838005, 40.80575305283603), (30.764447610214805, 40.810226206326156), (30.773800770797507, 40.814665698376835), (30.774051881482734, 40.814766860196016), (30.774313324307084, 40.81483713066985), (30.79022585045604, 40.818102160684624), (30.79053449063101, 40.8181434922337), (30.80124329748096, 40.81882838192597), (30.810599265020546, 40.82430993616578), (30.810814120682277, 40.82442076467049), (30.81103962709436, 40.824507901178016), (30.811273180819416, 40.82457033971056), (30.84426805023462, 40.83151616122415), (30.

catalog name: pneo 



2024-02-24 11:43:59,500 - 0 results returned.
2024-02-24 11:43:59,514 - Searching catalog with search_parameters: {'datetime': '2023-05-01T00:00:00Z/2023-09-30T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((30.561470275866256, 40.7576736463021), (30.71621035278162, 40.792097518301624), (30.738823082999087, 40.79860291607337), (30.747175018694804, 40.80533377950656), (30.747363026007253, 40.805469917762096), (30.747564058842183, 40.80558596160483), (30.747775977603993, 40.80568067597832), (30.747996526838005, 40.80575305283603), (30.764447610214805, 40.810226206326156), (30.773800770797507, 40.814665698376835), (30.774051881482734, 40.814766860196016), (30.774313324307084, 40.81483713066985), (30.79022585045604, 40.818102160684624), (30.79053449063101, 40.8181434922337), (30.80124329748096, 40.81882838192597), (30.810599265020546, 40.82430993616578), (30.810814120682277, 40.82442076467049), (30.81103962709436, 40.824507901178016), (30.811273180819416, 40.82457033971056)

catalog name: hexagon-na-30cm 



2024-02-24 11:43:59,928 - 0 results returned.
2024-02-24 11:43:59,928 - Searching catalog with search_parameters: {'datetime': '2023-05-01T00:00:00Z/2023-09-30T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((30.561470275866256, 40.7576736463021), (30.71621035278162, 40.792097518301624), (30.738823082999087, 40.79860291607337), (30.747175018694804, 40.80533377950656), (30.747363026007253, 40.805469917762096), (30.747564058842183, 40.80558596160483), (30.747775977603993, 40.80568067597832), (30.747996526838005, 40.80575305283603), (30.764447610214805, 40.810226206326156), (30.773800770797507, 40.814665698376835), (30.774051881482734, 40.814766860196016), (30.774313324307084, 40.81483713066985), (30.79022585045604, 40.818102160684624), (30.79053449063101, 40.8181434922337), (30.80124329748096, 40.81882838192597), (30.810599265020546, 40.82430993616578), (30.810814120682277, 40.82442076467049), (30.81103962709436, 40.824507901178016), (30.811273180819416, 40.82457033971056)

...

catalog name: eros-b 



2024-02-24 11:44:08,434 - 0 results returned.
2024-02-24 11:44:08,444 - Searching catalog with search_parameters: {'datetime': '2023-05-01T00:00:00Z/2023-09-30T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((30.561470275866256, 40.7576736463021), (30.71621035278162, 40.792097518301624), (30.738823082999087, 40.79860291607337), (30.747175018694804, 40.80533377950656), (30.747363026007253, 40.805469917762096), (30.747564058842183, 40.80558596160483), (30.747775977603993, 40.80568067597832), (30.747996526838005, 40.80575305283603), (30.764447610214805, 40.810226206326156), (30.773800770797507, 40.814665698376835), (30.774051881482734, 40.814766860196016), (30.774313324307084, 40.81483713066985), (30.79022585045604, 40.818102160684624), (30.79053449063101, 40.8181434922337), (30.80124329748096, 40.81882838192597), (30.810599265020546, 40.82430993616578), (30.810814120682277, 40.82442076467049), (30.81103962709436, 40.824507901178016), (30.811273180819416, 40.82457033971056)

catalog name: vexcel-weu-15cm-ortho 



2024-02-24 11:44:09,297 - 0 results returned.


catalog name: eros-c 



In [162]:
results_list = []

for i in catalog_names_array:
    search_parameters = catalog.construct_search_parameters(
        collections=[i], #name
        geometry = aoi,
        start_date=start_date_search,
        end_date=end_date_search,
        max_cloudcover=20,
        limit=10,
    )
    search_results_json = catalog.search((search_parameters), as_dataframe=False)
    log_messages = str(search_results_json).split('\n')
    for msg in log_messages:
        if "results returned." in msg:
            results_count = msg.split()[0]
            results_list.append({'Collection': i, 'Results': results_count})

# Create a DataFrame from the list
results_df = pd.DataFrame(results_list)
print(results_df)


2024-02-24 11:41:25,163 - Searching catalog with search_parameters: {'datetime': '2023-05-01T00:00:00Z/2023-09-30T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((30.561470275866256, 40.7576736463021), (30.71621035278162, 40.792097518301624), (30.738823082999087, 40.79860291607337), (30.747175018694804, 40.80533377950656), (30.747363026007253, 40.805469917762096), (30.747564058842183, 40.80558596160483), (30.747775977603993, 40.80568067597832), (30.747996526838005, 40.80575305283603), (30.764447610214805, 40.810226206326156), (30.773800770797507, 40.814665698376835), (30.774051881482734, 40.814766860196016), (30.774313324307084, 40.81483713066985), (30.79022585045604, 40.818102160684624), (30.79053449063101, 40.8181434922337), (30.80124329748096, 40.81882838192597), (30.810599265020546, 40.82430993616578), (30.810814120682277, 40.82442076467049), (30.81103962709436, 40.824507901178016), (30.811273180819416, 40.82457033971056), (30.84426805023462, 40.83151616122415), (30.

Empty DataFrame
Columns: []
Index: []


In [126]:
def search_catalog(collections, geometry, start_date, end_date, max_cloudcover=20, limit=10):
    """
    Search the catalog based on the specified parameters.
    
    Parameters:
    - collections (list of str): Names of the collections to search.
    - geometry (dict): Geometry of the area of interest.
    - start_date (str): Start date of the search (YYYY-MM-DD).
    - end_date (str): End date of the search (YYYY-MM-DD).
    - max_cloudcover (int): Maximum cloud cover percentage allowed.
    - limit (int): Maximum number of search results to return.
    
    Returns:
    - search_results_json (list): Search results in JSON format.
    - search_results_df (DataFrame): Search results in DataFrame format.
    """
    # Construct search parameters
    search_parameters = catalog.construct_search_parameters(
        collections=collections,
        geometry=aoi,
        start_date=start_date,
        end_date=end_date,
        max_cloudcover=max_cloudcover,
        limit=limit
    )
    
    # Search the catalog and retrieve results
    search_results_json = catalog.search(search_parameters, as_dataframe=False)
    search_results_df = catalog.search(search_parameters, as_dataframe=True)
    
    return search_results_json, search_results_df

for collection in catalog_names_array:
    search_results_json, search_results_df = search_catalog([collection], aoi, start_date, end_date, max_cloudcover, limit)
    
    # Process search results as needed
    print(f"Search results for collection '{collection}':")
    print("Search results in JSON format:", search_results_json)
    print("Search results in DataFrame format:")
    print(search_results_df)


2024-02-24 11:11:49,150 - Searching catalog with search_parameters: {'datetime': '2024-01-01T00:00:00Z/2024-12-31T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((30.561469902723548, 40.757675868297596), (30.71620979692899, 40.792099699651004), (30.73882202462908, 40.79860495285483), (30.747173606831108, 40.80533553140583), (30.747361802150873, 40.80547180579962), (30.747563036018636, 40.805587965686186), (30.747775166699206, 40.805682774774056), (30.74799593648245, 40.80575522400863), (30.764446826467374, 40.810228324914284), (30.773799805996664, 40.814667731027676), (30.774051167792575, 40.81476899400867), (30.774312872059745, 40.81483933475298), (30.7902253982087, 40.81810436476776), (30.790534347023847, 40.818145737648386), (30.80124262088376, 40.81883059325288), (30.810598127610525, 40.82431187750774), (30.81081319812792, 40.82442281684095), (30.81103893004642, 40.82451004048498), (30.8112727173252, 40.824572541456064), (30.844267586740404, 40.83151836296965), (30.8

Search results for collection 'pneo':
Search results in JSON format: {'type': 'FeatureCollection', 'features': [], 'bbox': (nan, nan, nan, nan)}
Search results in DataFrame format:
Empty GeoDataFrame
Columns: [geometry]
Index: []


2024-02-24 11:11:50,673 - 0 results returned.
2024-02-24 11:11:50,673 - Searching catalog with search_parameters: {'datetime': '2024-01-01T00:00:00Z/2024-12-31T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((30.561469902723548, 40.757675868297596), (30.71620979692899, 40.792099699651004), (30.73882202462908, 40.79860495285483), (30.747173606831108, 40.80533553140583), (30.747361802150873, 40.80547180579962), (30.747563036018636, 40.805587965686186), (30.747775166699206, 40.805682774774056), (30.74799593648245, 40.80575522400863), (30.764446826467374, 40.810228324914284), (30.773799805996664, 40.814667731027676), (30.774051167792575, 40.81476899400867), (30.774312872059745, 40.81483933475298), (30.7902253982087, 40.81810436476776), (30.790534347023847, 40.818145737648386), (30.80124262088376, 40.81883059325288), (30.810598127610525, 40.82431187750774), (30.81081319812792, 40.82442281684095), (30.81103893004642, 40.82451004048498), (30.8112727173252, 40.824572541456064), 

Search results for collection 'hexagon-na-30cm':
Search results in JSON format: {'type': 'FeatureCollection', 'features': [], 'bbox': (nan, nan, nan, nan)}
Search results in DataFrame format:
Empty GeoDataFrame
Columns: [geometry]
Index: []


2024-02-24 11:11:51,556 - 0 results returned.
2024-02-24 11:11:51,570 - Searching catalog with search_parameters: {'datetime': '2024-01-01T00:00:00Z/2024-12-31T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((30.561469902723548, 40.757675868297596), (30.71620979692899, 40.792099699651004), (30.73882202462908, 40.79860495285483), (30.747173606831108, 40.80533553140583), (30.747361802150873, 40.80547180579962), (30.747563036018636, 40.805587965686186), (30.747775166699206, 40.805682774774056), (30.74799593648245, 40.80575522400863), (30.764446826467374, 40.810228324914284), (30.773799805996664, 40.814667731027676), (30.774051167792575, 40.81476899400867), (30.774312872059745, 40.81483933475298), (30.7902253982087, 40.81810436476776), (30.790534347023847, 40.818145737648386), (30.80124262088376, 40.81883059325288), (30.810598127610525, 40.82431187750774), (30.81081319812792, 40.82442281684095), (30.81103893004642, 40.82451004048498), (30.8112727173252, 40.824572541456064), 

...

Search results for collection 'eros-b':
Search results in JSON format: {'type': 'FeatureCollection', 'features': [], 'bbox': (nan, nan, nan, nan)}
Search results in DataFrame format:
Empty GeoDataFrame
Columns: [geometry]
Index: []


2024-02-24 11:12:08,242 - 0 results returned.
2024-02-24 11:12:08,244 - Searching catalog with search_parameters: {'datetime': '2024-01-01T00:00:00Z/2024-12-31T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((30.561469902723548, 40.757675868297596), (30.71620979692899, 40.792099699651004), (30.73882202462908, 40.79860495285483), (30.747173606831108, 40.80533553140583), (30.747361802150873, 40.80547180579962), (30.747563036018636, 40.805587965686186), (30.747775166699206, 40.805682774774056), (30.74799593648245, 40.80575522400863), (30.764446826467374, 40.810228324914284), (30.773799805996664, 40.814667731027676), (30.774051167792575, 40.81476899400867), (30.774312872059745, 40.81483933475298), (30.7902253982087, 40.81810436476776), (30.790534347023847, 40.818145737648386), (30.80124262088376, 40.81883059325288), (30.810598127610525, 40.82431187750774), (30.81081319812792, 40.82442281684095), (30.81103893004642, 40.82451004048498), (30.8112727173252, 40.824572541456064), 

Search results for collection 'vexcel-weu-15cm-ortho':
Search results in JSON format: {'type': 'FeatureCollection', 'features': [], 'bbox': (nan, nan, nan, nan)}
Search results in DataFrame format:
Empty GeoDataFrame
Columns: [geometry]
Index: []


2024-02-24 11:12:09,582 - 0 results returned.
2024-02-24 11:12:09,583 - Searching catalog with search_parameters: {'datetime': '2024-01-01T00:00:00Z/2024-12-31T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((30.561469902723548, 40.757675868297596), (30.71620979692899, 40.792099699651004), (30.73882202462908, 40.79860495285483), (30.747173606831108, 40.80533553140583), (30.747361802150873, 40.80547180579962), (30.747563036018636, 40.805587965686186), (30.747775166699206, 40.805682774774056), (30.74799593648245, 40.80575522400863), (30.764446826467374, 40.810228324914284), (30.773799805996664, 40.814667731027676), (30.774051167792575, 40.81476899400867), (30.774312872059745, 40.81483933475298), (30.7902253982087, 40.81810436476776), (30.790534347023847, 40.818145737648386), (30.80124262088376, 40.81883059325288), (30.810598127610525, 40.82431187750774), (30.81081319812792, 40.82442281684095), (30.81103893004642, 40.82451004048498), (30.8112727173252, 40.824572541456064), 

Search results for collection 'eros-c':
Search results in JSON format: {'type': 'FeatureCollection', 'features': [], 'bbox': (nan, nan, nan, nan)}
Search results in DataFrame format:
Empty GeoDataFrame
Columns: [geometry]
Index: []


In [127]:
def search_catalog(collections, geometry, start_date, end_date, max_cloudcover=20, limit=10):
    """
    Search the catalog based on the specified parameters.
    
    Parameters:
    - collections (list of str): Names of the collections to search.
    - geometry (dict): Geometry of the area of interest.
    - start_date (str): Start date of the search (YYYY-MM-DD).
    - end_date (str): End date of the search (YYYY-MM-DD).
    - max_cloudcover (int): Maximum cloud cover percentage allowed.
    - limit (int): Maximum number of search results to return.
    
    Returns:
    - search_results_df (DataFrame): Search results in DataFrame format.
    """
    # Construct search parameters
    search_parameters = catalog.construct_search_parameters(
        collections=collections,
        geometry=aoi,
        start_date=start_date,
        end_date=end_date,
        max_cloudcover=max_cloudcover,
        limit=limit
    )
    
    # Search the catalog and retrieve results
    search_results_df = catalog.search(search_parameters, as_dataframe=True)
    
    return search_results_df

# Example usage:
search_results_df_list = []
for collection in catalog_names_array:
    search_results_df = search_catalog([collection], aoi, start_date, end_date, max_cloudcover, limit)
    search_results_df_list.append(search_results_df)

# Concatenate all DataFrames into a single DataFrame
combined_search_results_df = pd.concat(search_results_df_list, ignore_index=True)


2024-02-24 11:13:30,795 - Searching catalog with search_parameters: {'datetime': '2024-01-01T00:00:00Z/2024-12-31T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((30.561469902723548, 40.757675868297596), (30.71620979692899, 40.792099699651004), (30.73882202462908, 40.79860495285483), (30.747173606831108, 40.80533553140583), (30.747361802150873, 40.80547180579962), (30.747563036018636, 40.805587965686186), (30.747775166699206, 40.805682774774056), (30.74799593648245, 40.80575522400863), (30.764446826467374, 40.810228324914284), (30.773799805996664, 40.814667731027676), (30.774051167792575, 40.81476899400867), (30.774312872059745, 40.81483933475298), (30.7902253982087, 40.81810436476776), (30.790534347023847, 40.818145737648386), (30.80124262088376, 40.81883059325288), (30.810598127610525, 40.82431187750774), (30.81081319812792, 40.82442281684095), (30.81103893004642, 40.82451004048498), (30.8112727173252, 40.824572541456064), (30.844267586740404, 40.83151836296965), (30.8

In [128]:
combined_search_results_df 

,geometry,id,constellation,collection,providerName,up42:usageType,providerProperties,sceneId,producer,acquisitionDate,start_datetime,end_datetime,cloudCoverage,resolution,deliveryTime
0,"POLYGON ((32.46374 40.97894, 32.42784 40.97887...",c2986682-6872-4f9c-9e70-626ca278a933,phr,phr,oneatlas,[DATA],"{'qualified': False, 'acquisitionStation': 'SV...",c2986682-6872-4f9c-9e70-626ca278a933,airbus,2024-02-06T08:48:40Z,None,None,19.6,0.5,HOURS


In [ ]:
search_results_df

In [ ]:
search_results_df
image_id = search_results_df['id'].values
image_id

In [ ]:
catalog.download_quicklooks(
    image_ids=image_id,
    collection='pneo',
    output_directory="./quicklooks/",
)

Export search results to a vector file that can be ingested into a GIS or plotted in jupyter
notebook

3. Estimate the cost

In [ ]:
catalog.map_quicklooks(scenes=search_results_df, aoi=aoi)

In [ ]:
# Place and track the order of your selected scene
order_parameters = catalog.construct_order_parameters(
    data_product_id=data_product_id, image_id=search_results.id[0])
catalog.estimate_order(order_parameters)
order = catalog.place_order(order_parameters, track_status=True)

In [ ]:
https://github.com/up42/documentation-notebooks/blob/main/processing/requests_pansharpening.ipynb

In [ ]:
# Stream cloud-native files directly for your use case
asset = up42.initialize_order(order_id=order.order_id).get_assets()[0]
stac_items = asset.stac_items
asset.get_stac_asset_url(stac_asset=stac_items[0].assets.get("b02.tiff"))

coverage in square kilometers and in percentage

suggestions for the areas that are not covered by API integrated catalog data.